In [5]:
import os
import numpy as np
from matplotlib import pyplot
from scipy import optimize
from scipy.io import loadmat
import utils

grader = utils.Grader()

%matplotlib inline

In [150]:
data = loadmat( os.path.join('Data', 'ex4data1.mat') )
X,y = data['X'] , data['y'].ravel()
m = y.size
y[y == 10] = 0
Y = np.zeros((m, output_layer_units), dtype='uint8') 
for i, row in enumerate(Y):
        Y[i,y[i]-1] = 1 #Creating a new 1D array filling it with one initially changing it from 0-9 

In [7]:
input_layer_size = 400 #also features
hidden_layer_units = 25
output_layer_units = 10

weights = loadmat('ex4_orig_octave/ex4weights.mat')

# Theta1 has size 25 x 401
# Theta2 has size 10 x 26
Theta1, Theta2 = weights['Theta1'], weights['Theta2']

print(Theta1.shape)
print(X.shape)
print(Theta1)
all_theta = np.concatenate( [Theta1.ravel(), Theta2.ravel()] ) #Rolling the Parameters

def sigmoid(z):
    return (1/(1+np.exp(-1*(z) )))

(25, 401)
(5000, 400)
[[-2.25623899e-02 -1.05624163e-08  2.19414684e-09 ... -1.30529929e-05
  -5.04175101e-06  2.80464449e-09]
 [-9.83811294e-02  7.66168682e-09 -9.75873689e-09 ... -5.60134007e-05
   2.00940969e-07  3.54422854e-09]
 [ 1.16156052e-01 -8.77654466e-09  8.16037764e-09 ... -1.20951657e-04
  -2.33669661e-06 -7.50668099e-09]
 ...
 [-1.83220638e-01 -8.89272060e-09 -9.81968100e-09 ...  2.35311186e-05
  -3.25484493e-06  9.02499060e-09]
 [-7.02096331e-01  3.05178374e-10  2.56061008e-09 ... -8.61759744e-04
   9.43449909e-05  3.83761998e-09]
 [-3.50933229e-01  8.85876862e-09 -6.57515140e-10 ... -1.80365926e-06
  -8.14464807e-06  8.79454531e-09]]


In [152]:
def sigmoidGradient(z):
    return sigmoid(z)*(1-sigmoid(z))
def nnCostFunction(all_theta, input_layer_size, hidden_layer_units, output_layer_units, X,y , lambda_):
    Theta1 = np.reshape(all_theta[:hidden_layer_units * (input_layer_size + 1)],
                        (hidden_layer_units, (input_layer_size + 1)))

    Theta2 = np.reshape(all_theta[(hidden_layer_units * (input_layer_size + 1)):],
                        (output_layer_units, (hidden_layer_units + 1)))
    
  
    
    a_1 = X.copy()
    emne1, emne2 = Theta1.shape
    Theta1_grad = np.zeros((emne1+1,emne2))
    Theta2_grad = np.zeros(Theta2.shape)
    a_1 = np.hstack(( np.ones((m,1)),a_1 ))
    a_2 = sigmoid(a_1.dot(Theta1.T))
    
    a_2 = a_2.T
    
    a_2 = np.vstack((np.ones(m), a_2))
    
    a_3 = sigmoid(Theta2.dot(a_2) )
    #print( Y.dot( np.log( a_3) ) + (1-Y).dot( np.log( 1-a_3 ) ))
    J = -1/m * np.trace( Y.dot( np.log( a_3) ) + (1-Y).dot( np.log( 1-a_3 ) ) ) + (lambda_/(2*m))*(np.sum( Theta1[:,1:]*Theta1[:,1:]   ) + np.sum( Theta2[:,1:]*Theta2[:,1:]) )
    for t in range(m): #This is forward propagation on a single data set
        
        z2 = Theta1.dot(a_1[t][:])
        a2 = sigmoid(z2)
        
        a2 = np.hstack(( np.ones((1)),a2 ))
       
        #print(a2)
        z3 = Theta2.dot(a2)
        a3 = sigmoid(z3)
        
        #Step 2
        delta_3 = a3 - Y[:][t]
        
        z2 = np.hstack( (np.ones(1),z2) )
        
        #step3
        delta_3 = np.reshape(delta_3, ((10,1)) )
        
        z2 = np.reshape( z2, ((26,1)) )
        delta_2 = (Theta2).T.dot(delta_3)*sigmoid(z2)
        
        
        
        #step4
        a2 = np.reshape(a2, ((1,26)) )
        
        
        
        
        Theta2_grad = Theta2_grad + (delta_3).dot(a2)
        a1 = np.reshape( a_1[t][:], ((1,401)) )
        delta_2 = np.reshape(delta_2, ((26,1)) )
        Theta1_grad  = Theta1_grad + delta_2.dot(a1)
        
    
    Theta1_grad = (1/m)*Theta1_grad
    Theta2_grad = (1/m)*Theta1_grad
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])                                                                                               
    return J,grad
    
    
    
    
    
lambda_ = 0
print( all_theta.shape )
J, grad= nnCostFunction(all_theta, input_layer_size, hidden_layer_units, output_layer_units, X, y, 
                     lambda_)

print('Cost at parameters (loaded from ex4weights): %.6f ' % J)
print('The cost should be about                   : 0.287629.')

(10285,)
Cost at parameters (loaded from ex4weights): 0.287629 
The cost should be about                   : 0.287629.


In [11]:
# Weight regularization parameter (we set this to 1 here).
lambda_ = 1
J= nnCostFunction(all_theta, input_layer_size, hidden_layer_units, output_layer_units, X, y, 
                     lambda_)

print('Cost at parameters (loaded from ex4weights): %.6f' % J)
print('This value should be about                 : 0.383770.')


(10, 5000)
Cost at parameters (loaded from ex4weights): 0.383770
This value should be about                 : 0.383770.


In [146]:

    
z = np.array([-1, -0.5, 0, 0.5, 1])
g = sigmoidGradient(z)
print('Sigmoid gradient evaluated at [-1 -0.5 0 0.5 1]:\n  ')
print(g)    

Sigmoid gradient evaluated at [-1 -0.5 0 0.5 1]:
  
[0.19661193 0.23500371 0.25       0.23500371 0.19661193]


In [135]:
def randInitializeWeights(L_in, L_out, epsilon_init = 0.12):
    theta = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init
    return theta


theta1 = randInitializeWeights(input_layer_size, hidden_layer_units)
theta2 = randInitializeWeights(hidden_layer_units, output_layer_units)

unroll = np.concatenate( [theta1.ravel(), theta2.ravel()] )

In [136]:
J, grad = nnCostFunction(unroll, input_layer_size, hidden_layer_units, output_layer_units, X, y, 
                     lambda_)


In [158]:
options = {'maxiter' : 100}
lambda_ = 1
costFunction = lambda p : nnCostFunction(p, input_layer_size,hidden_layer_units, output_layer_units, X, y, 
                     lambda_)
res = optimize.minimize(costFunction, unroll, jac= True, method = 'TNC' , options = options )

nn_params = res.x
Theta1 = np.reshape(nn_params[:hidden_layer_units * (input_layer_size + 1)],
                    (hidden_layer_units, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_units * (input_layer_size + 1)):],
                    (output_layer_units, (hidden_layer_units + 1)))

pred = utils.predict(Theta1, Theta2, X)

ValueError: tnc: invalid gradient vector from minimized function.